In [ ]:
pip install flask fastapi uvicorn scikit-learn


In [ ]:
import pickle
from flask import Flask, request, jsonify
from fastapi import FastAPI
from pydantic import BaseModel


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Load the Iris dataset
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

# Train a Random Forest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Save the model
with open("iris_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("Model trained and saved successfully!")


In [ ]:
# Load the model
with open("iris_model.pkl", "rb") as f:
    model = pickle.load(f)

# Initialize Flask app
flask_app = Flask(__name__)

@flask_app.route("/predict", methods=["POST"])
def predict():
    # Get JSON data
    data = request.json
    features = data["features"]

    # Make prediction
    prediction = model.predict([features])
    predicted_class = iris.target_names[prediction[0]]
    
    return jsonify({"prediction": predicted_class})

if __name__ == "__main__":
    flask_app.run(debug=True, port=5000)


In [ ]:
curl -X POST -H "Content-Type: application/json" -d '{"features": [5.1, 3.5, 1.4, 0.2]}' http://127.0.0.1:5000/predict


In [ ]:
# Initialize FastAPI app
fastapi_app = FastAPI()

# Request model for FastAPI
class RequestModel(BaseModel):
    features: list

@fastapi_app.post("/predict")
def predict(request: RequestModel):
    # Get features from request
    features = request.features

    # Make prediction
    prediction = model.predict([features])
    predicted_class = iris.target_names[prediction[0]]
    
    return {"prediction": predicted_class}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(fastapi_app, host="127.0.0.1", port=8000)


In [ ]:
curl -X POST -H "Content-Type: application/json" -d '{"features": [5.1, 3.5, 1.4, 0.2]}' http://127.0.0.1:8000/predict


In [ ]:
FROM python:3.9
WORKDIR /app
COPY . /app
RUN pip install -r requirements.txt
CMD ["python", "app.py"]  # For Flask, replace `app.py` with your file name


In [ ]:
docker build -t ml-api .
docker run -p 5000:5000 ml-api


In [ ]:
heroku create
git push heroku main
heroku open
